In [8]:
import os
import sys
from dotenv import load_dotenv
sys.path.append(os.path.abspath('..'))

load_dotenv()

DB_CONFIG = {
    "dbname": os.getenv("PG_NAME"),
    "user": os.getenv("PG_USER"),
    "password": os.getenv("PG_PASSWORD"),
    "host": os.getenv("PG_HOST"),  # or your db host
    "port": os.getenv("PG_PORT", "5432"),  # default postgres port
}

In [23]:
import psycopg2
from sqlalchemy import create_engine, text
import pandas as pd
import src.agent.adapters.tools as tools

In [24]:
DATABASE_URL = (
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
)

In [25]:
try:
    engine = create_engine(DATABASE_URL)
    print("SQLAlchemy engine created successfully.")
except Exception as e:
    print(f"Error creating SQLAlchemy engine: {e}")
    engine = None

SQLAlchemy engine created successfully.


In [26]:
def execute_sql(engine, sql_statement, params=None):
    """
    Executes a SELECT SQL query and returns the result as a Pandas DataFrame.
    This is the officially supported way.
    """
    if not engine:
        print("Engine not available. Cannot execute query.")
        return None

    try:
        print(f"Executing SQL: {sql_statement[:60]}...")
        # Pandas reads directly from the SQLAlchemy engine
        df = pd.read_sql_query(sql=text(sql_statement), con=engine, params=params)
        print(f"Query returned a DataFrame with {len(df)} rows.")
        return df
    except Exception as e:
        print(f"Error executing query to DataFrame: {e}")
        return None

In [20]:
statement = """SELECT
  COALESCE(SUM(od.quantity * od.final_unit_price), 0) AS total_revenue
FROM
  orders AS o
JOIN
  order_details AS od ON o.order_id = od.order_id
WHERE
  -- Filter for orders shipped today or yesterday
  o.shipped_date >= CURRENT_DATE - INTERVAL '1 year';
"""

In [27]:
result = execute_sql(engine, statement)

Executing SQL: SELECT
  COALESCE(SUM(od.quantity * od.final_unit_price), 0)...
Query returned a DataFrame with 1 rows.


In [28]:
result

,total_revenue
0,8.825503e+08
